In [52]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

#### Reading dataset

In [53]:

stock = pd.read_csv("Stock_Dataa.csv",encoding='ISO-8859-1')
stock.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,2000-01-05,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,...,South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,2000-01-06,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,...,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,2000-01-07,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,...,Most everywhere: UDIs,Most wanted: Chloe lunettes,Return of the cane 'completely off the agenda',From Sleepy Hollow to Greeneland,Blunkett outlines vision for over 11s,"Embattled Dobson attacks 'play now, pay later'...",Doom and the Dome,What is the north-south divide?,Aitken released from jail,Gone aloft


In [54]:
stock.tail()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
4096,2016-06-27,0,Barclays and RBS shares suspended from trading...,Pope says Church should ask forgiveness from g...,Poland 'shocked' by xenophobic abuse of Poles ...,"There will be no second referendum, cabinet ag...","Scotland welcome to join EU, Merkel ally says",Sterling dips below Friday's 31-year low amid ...,No negative news about South African President...,Surge in Hate Crimes in the U.K. Following U.K...,...,German lawyers to probe Erdogan over alleged w...,"Boris Johnson says the UK will continue to ""in...",Richard Branson is calling on the UK governmen...,Turkey 'sorry for downing Russian jet',Edward Snowden lawyer vows new push for pardon...,Brexit opinion poll reveals majority don't wan...,"Conservative MP Leave Campaigner: ""The leave c...","Economists predict UK recession, further weake...","New EU 'superstate plan by France, Germany: Cr...",Pakistani clerics declare transgender marriage...
4097,2016-06-28,1,"2,500 Scientists To Australia: If You Want To ...","The personal details of 112,000 French police ...",S&amp;P cuts United Kingdom sovereign credit r...,Huge helium deposit found in Africa,CEO of the South African state broadcaster qui...,"Brexit cost investors $2 trillion, the worst o...",Hong Kong democracy activists call for return ...,Brexit: Iceland president says UK can join 'tr...,...,"US, Canada and Mexico pledge 50% of power from...",There is increasing evidence that Australia is...,"Richard Branson, the founder of Virgin Group, ...","37,000-yr-old skull from Borneo reveals surpri...",Palestinians stone Western Wall worshipers; po...,Jean-Claude Juncker asks Farage: Why are you h...,"""Romanians for Remainians"" offering a new home...",Brexit: Gibraltar in talks with Scotland to st...,8 Suicide Bombers Strike Lebanon,Mexico's security forces routinely use 'sexual...
4098,2016-06-29,1,Explosion At Airport In Istanbul,Yemeni former president: Terrorism is the offs...,UK must accept freedom of movement to access E...,Devastated: scientists too late to captive bre...,British Labor Party leader Jeremy Corbyn loses...,A Muslim Shop in the UK Was Just Firebombed Wh...,Mexican Authorities Sexually Torture Women in ...,UK shares and pound continue to recover,...,"Escape Tunnel, Dug by Hand, Is Found at Holoca...",The land under Beijing is sinking by as much a...,Car bomb and Anti-Islamic attack on Mosque in ...,Emaciated lions in Taiz Zoo are trapped in blo...,Rupert Murdoch describes Brexit as 'wonderful'...,More than 40 killed in Yemen suicide attacks,Google Found Disastrous Symantec and Norton Vu...,Extremist violence on the rise in Germany: Dom...,BBC News: Labour MPs pass Corbyn no-confidence...,Tiny New Zealand town with 'too many jobs' lau...
4099,2016-06-30,1,Jamaica proposes marijuana dispensers for tour...,Stephen Hawking says pollution and 'stupidity'...,Boris Johnson says he will not run for Tory pa...,Six gay men in Ivory Coast were abused and for...,Switzerland denies citizenship to Muslim immig...,Palestinian terrorist stabs israeli teen girl ...,Puerto Rico will default on $1 billion of debt...,Republic of Ireland fans to be awarded medal f...,...,Googles free wifi at Indian railway stations i...,Mounting evidence suggests 'hobbits' were wipe...,The men who carried out Tuesday's terror attac...,Calls to suspend Saudi Arabia from UN Human Ri...,More Than 100 Nobel Laureates Call Out Greenpe...,British pedophile sentenced to 85 years in US ...,"US permitted 1,200 offshore fracks in Gulf of ...",We will be swimming in ridicule - French beach...,UEFA says no minutes of silence for Istanbul v...,Law Enforcement Sources: Gun Used in Paris Ter...
4100,2016-07-01,1,A 117-year-old woman in Mexico City finally re...,IMF chief backs Athens as permanent Olympic host,"The president of France says if Brexit won, so...",British Man Who Must Give Police 24 Hours' Not...,100+ Nobel laureates 

#### Dividing the original data into train and test dataset

In [55]:
traindata=stock[stock['Date']<'20150101']
testdata=stock[stock['Date']>'20141231']

In [56]:
textd=traindata.iloc[:,2:27]


In [57]:
list=[]
for i in range((25)):
    list.append(i)
#list
textd.columns=list

In [58]:
#text.iloc[:,:]
headline=[]
for i in range(0,len(traindata.index)):
    headline.append(' '.join(str(x) for x in textd.iloc[i,:]))
headline[0]

"A 'hindrance to operations': extracts from the leaked reports Scorecard Hughes' instant hit buoys Blues Jack gets his skates on at ice-cold Alex Chaos as Maracana builds up for United Depleted Leicester prevail as Elliott spoils Everton's party Hungry Spurs sense rich pickings Gunners so wide of an easy target Derby raise a glass to Strupar's debut double Southgate strikes, Leeds pay the penalty Hammers hand Robson a youthful lesson Saints party like it's 1999 Wear wolves have turned into lambs Stump mike catches testy Gough's taunt Langer escapes to hit 167 Flintoff injury piles on woe for England Hunters threaten Jospin with new battle of the Somme Kohl's successor drawn into scandal The difference between men and women Sara Denver, nurse turned solicitor Diana's landmine crusade put Tories in a panic Yeltsin's resignation caught opposition flat-footed Russian roulette Sold out Recovering a title"

#### Removing all the unwanted words and lower the letter

In [35]:
lemmatizer=  WordNetLemmatizer()
Array1= []
for i in range(0,len(traindata.index)):
    text = re.sub('[^a-zA-Z]', ' ', headline[i])
    text = text.lower()
    #text = text.split()
    #word = [lemmatizer.lemmatize(word) for word in text if not word in set(stopwords.words('english'))]
    #text = ' '.join(text)
    Array1.append(text)
   

In [59]:
tfidf=TfidfVectorizer()
X=tfidf.fit_transform(headline).toarray()

In [60]:
X_train = X
y_train=traindata['Label']

## Predicting for the Test Dataset

In [64]:

headline2= []
for row in range(0,len(testdata.index)):
    headline2.append(' '.join(str(x) for x in testdata.iloc[row,2:27]))
X2 = tfidf.transform(headline2).toarray()


In [65]:
X_test = X2
y_test=testdata['Label']

#### Multinomial Naive Bayes

In [68]:
spam_detect_model = MultinomialNB().fit(X_train, y_train)
y_pred=spam_detect_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average="binary")
precision = precision_score(y_test, y_pred , average="binary")
f1 = f1_score(y_test, y_pred, average="binary")
print(confusion_matrix(y_test, y_pred))
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)
from sklearn import metrics
# save confusion matrix and slice into four pieces
confusion=metrics.confusion_matrix(y_test, y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print ("TP= ",TP)
print ("TN= ",TN)
print ("FP= ",FP)
print ("FN= ",FN)
sensitivity = TP / float(FN + TP)
print("sensitivity= ",sensitivity)
print("recall: ", metrics.recall_score(y_test, y_pred))
specificity = TN / (TN + FP)
print("Specificity= ",specificity)
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
print ("roc_auc_score ",roc_auc_score(y_test, y_pred))


[[  0 186]
 [  0 192]]
accuracy
0.508
precision
0.508
racall
1.000
f1score
0.674
TP=  192
TN=  0
FP=  186
FN=  0
sensitivity=  1.0
recall:  1.0
Specificity=  0.0
roc_auc_score  0.5


#### Logistic Regression

In [69]:
spam_detect_model = LogisticRegression().fit(X_train, y_train)
y_pred=spam_detect_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average="binary")
precision = precision_score(y_test, y_pred , average="binary")
f1 = f1_score(y_test, y_pred, average="binary")
print(confusion_matrix(y_test, y_pred))
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)
from sklearn import metrics
# save confusion matrix and slice into four pieces
confusion=metrics.confusion_matrix(y_test, y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print ("TP= ",TP)
print ("TN= ",TN)
print ("FP= ",FP)
print ("FN= ",FN)
sensitivity = TP / float(FN + TP)
print("sensitivity= ",sensitivity)
print("recall: ", metrics.recall_score(y_test, y_pred))
specificity = TN / (TN + FP)
print("Specificity= ",specificity)
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
print ("roc_auc_score ",roc_auc_score(y_test, y_pred))

C:\Users\TANZIL\Anaconda3\Anacanda_python\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[[126  60]
 [ 15 177]]
accuracy
0.802
precision
0.747
racall
0.922
f1score
0.825
TP=  177
TN=  126
FP=  60
FN=  15
sensitivity=  0.921875
recall:  0.921875
Specificity=  0.6774193548387096
roc_auc_score  0.7996471774193549


#### Gaussuian Naive Bayes

In [70]:
spam_detect_model = GaussianNB().fit(X_train, y_train)
y_pred=spam_detect_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average="binary")
precision = precision_score(y_test, y_pred , average="binary")
f1 = f1_score(y_test, y_pred, average="binary")
print(confusion_matrix(y_test, y_pred))
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)
from sklearn import metrics
# save confusion matrix and slice into four pieces
confusion=metrics.confusion_matrix(y_test, y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print ("TP= ",TP)
print ("TN= ",TN)
print ("FP= ",FP)
print ("FN= ",FN)
sensitivity = TP / float(FN + TP)
print("sensitivity= ",sensitivity)
print("recall: ", metrics.recall_score(y_test, y_pred))
specificity = TN / (TN + FP)
print("Specificity= ",specificity)
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
print ("roc_auc_score ",roc_auc_score(y_test, y_pred))

[[152  34]
 [ 33 159]]
accuracy
0.823
precision
0.824
racall
0.828
f1score
0.826
TP=  159
TN=  152
FP=  34
FN=  33
sensitivity=  0.828125
recall:  0.828125
Specificity=  0.8172043010752689
roc_auc_score  0.8226646505376344


#### Decision Tree

In [71]:
spam_detect_model = DecisionTreeClassifier().fit(X_train, y_train)
y_pred=spam_detect_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average="binary")
precision = precision_score(y_test, y_pred , average="binary")
f1 = f1_score(y_test, y_pred, average="binary")
print(confusion_matrix(y_test, y_pred))
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)
from sklearn import metrics
# save confusion matrix and slice into four pieces
confusion=metrics.confusion_matrix(y_test, y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print ("TP= ",TP)
print ("TN= ",TN)
print ("FP= ",FP)
print ("FN= ",FN)
sensitivity = TP / float(FN + TP)
print("sensitivity= ",sensitivity)
print("recall: ", metrics.recall_score(y_test, y_pred))
specificity = TN / (TN + FP)
print("Specificity= ",specificity)
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
print ("roc_auc_score ",roc_auc_score(y_test, y_pred))

[[158  28]
 [ 35 157]]
accuracy
0.833
precision
0.849
racall
0.818
f1score
0.833
TP=  157
TN=  158
FP=  28
FN=  35
sensitivity=  0.8177083333333334
recall:  0.8177083333333334
Specificity=  0.8494623655913979
roc_auc_score  0.8335853494623657


#### Adaboost

In [ ]:
spam_detect_model =AdaBoostClassifier(n_estimators=100).fit(X_train, y_train)
y_pred=spam_detect_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average="binary")
precision = precision_score(y_test, y_pred , average="binary")
f1 = f1_score(y_test, y_pred, average="binary")
print(confusion_matrix(y_test, y_pred))
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)
from sklearn import metrics
# save confusion matrix and slice into four pieces
confusion=metrics.confusion_matrix(y_test, y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print ("TP= ",TP)
print ("TN= ",TN)
print ("FP= ",FP)
print ("FN= ",FN)
sensitivity = TP / float(FN + TP)
print("sensitivity= ",sensitivity)
print("recall: ", metrics.recall_score(y_test, y_pred))
specificity = TN / (TN + FP)
print("Specificity= ",specificity)
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
print ("roc_auc_score ",roc_auc_score(y_test, y_pred))

#### Random Forest Classifier

In [67]:
spam_detect_model =RandomForestClassifier(n_estimators=100).fit(X_train, y_train)
y_pred=spam_detect_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average="binary")
precision = precision_score(y_test, y_pred , average="binary")
f1 = f1_score(y_test, y_pred, average="binary")
print(confusion_matrix(y_test, y_pred))
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)
from sklearn import metrics
# save confusion matrix and slice into four pieces
confusion=metrics.confusion_matrix(y_test, y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print ("TP= ",TP)
print ("TN= ",TN)
print ("FP= ",FP)
print ("FN= ",FN)
sensitivity = TP / float(FN + TP)
print("sensitivity= ",sensitivity)
print("recall: ", metrics.recall_score(y_test, y_pred))
specificity = TN / (TN + FP)
print("Specificity= ",specificity)
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
print ("roc_auc_score ",roc_auc_score(y_test, y_pred))

[[147  39]
 [ 19 173]]
accuracy
0.847
precision
0.816
racall
0.901
f1score
0.856
TP=  173
TN=  147
FP=  39
FN=  19
sensitivity=  0.9010416666666666
recall:  0.9010416666666666
Specificity=  0.7903225806451613
roc_auc_score  0.8456821236559139
